In [122]:
from toolz import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
%matplotlib widget


import pathlib
from lenses import lens

from collections import Counter

import re
import nltk
import pymorphy2
import fasttext.util

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [35]:
nltk.download('popular')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger_ru')
nltk.download('tagsets')
nltk.download('stopwords')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\aakomlev\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\aakomlev\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\aakomlev\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\aakomlev\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\aakomlev\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to

True

In [36]:
lmap = compose(list, map)
ltake = compose(list, take)

In [37]:
def map_df_ingredients_element(func, df):
    df = df.copy()
    df['ingredients'] = df['ingredients'].map(lambda v: lmap(func, v))
    return df

In [38]:
data_path = pathlib.Path('data')

## Загружаем датафрейм с данными

In [39]:
df = pd.read_json(data_path.joinpath('ready_dataframe.json'))
df = map_df_ingredients_element(compose(str.lower, str.strip), df)
df

,id,title,course,cuisine,ingredients
0,28195,Банановое мороженое с корицей,Выпечка и десерты,Карибская кухня,"[бананы, лимонный сок, молотая корица, ванильн..."
1,28141,Ананасово-кокосовый шербет,Выпечка и десерты,Карибская кухня,"[сахар, кокосовое молоко, корень имбиря, анана..."
2,27929,"Десерт из манго, клубники и текилы",Выпечка и десерты,Карибская кухня,"[клубника, манго, сахар, тертая цедра лайма, т..."
3,28192,Ананас с соусом из манго и рома,Выпечка и десерты,Карибская кухня,"[манго, темный ром, сок лайма, сахар, тертая ц..."
4,28463,Клубничный соус с текилой,Выпечка и десерты,Карибская кухня,"[клубника, сахарная пудра, сок лайма, текила, ..."
...,...,...,...,...,...
41433,43380,Постный борщ с фасолью,Супы,Украинская кухня,"[белая фасоль, овощной бульон, свекла, картофе..."
41434,80446,Суп энгамат,Супы,Шведская кухня,"[цветная капуста, морковь, картофель, лук-поре..."
41435,136820,Гороховый суп с блинчиками,Супы,Шведская кухня,"[горох, репчатый лук, гвоздика, свиная рулька,..."
41436,18014,Суп из брокколи и кростини с сыром бри,Супы,Шведская кухня,"[оливковое масло, репчатый лук, овощной бульон..."


## Работа с ингридиентами

Удобнее работать с листом листов:

In [40]:
ingredients = list(df['ingredients'])
# TODO !!!!!!!!! Временно берем только первые несколько записей 
ingredients = ingredients[:20]
ingredients[:3]

[['бананы', 'лимонный сок', 'молотая корица', 'ванильное мороженое'],
 ['сахар', 'кокосовое молоко', 'корень имбиря', 'ананас', 'сок лайма'],
 ['клубника',
  'манго',
  'сахар',
  'тертая цедра лайма',
  'текила',
  'сок лайма',
  'апельсиновый ликер']]

### Самые часто встречающиеся слова в ингридиентах

In [41]:
ingredients_counter = Counter()
for lst in ingredients:
    ingredients_counter.update(lst)
# first_ingresients, _ = zip(*ingredients_counter.most_common(10))
# first_ingresients = list(first_ingresients)
# first_ingresients
ingredients_counter.most_common(10)

[('сахар', 15),
 ('пшеничная мука', 14),
 ('куриное яйцо', 11),
 ('сливочное масло', 9),
 ('соль', 7),
 ('сахарная пудра', 5),
 ('сок лайма', 4),
 ('сметана', 3),
 ('сода', 3),
 ('корица', 3)]

In [42]:
morph = pymorphy2.MorphAnalyzer()
def lemmatize(word):
    return morph.parse(word)[0].normal_form

it = lmap(compose(tuple, nltk.word_tokenize), ingredients_counter)
it = lens.Each().Each().modify(lemmatize)(it)
it[:10]

[('банан',),
 ('лимонный', 'сок'),
 ('молотый', 'корица'),
 ('ванильный', 'мороженое'),
 ('сахар',),
 ('кокосовый', 'молоко'),
 ('корень', 'имбирь'),
 ('ананас',),
 ('сок', 'лайм'),
 ('клубника',)]

### Токенизация

In [43]:
ingredients_tokenized = lens.Each().Each().modify(nltk.word_tokenize)(ingredients)
ingredients_tokenized[:4]

[[['бананы'],
  ['лимонный', 'сок'],
  ['молотая', 'корица'],
  ['ванильное', 'мороженое']],
 [['сахар'],
  ['кокосовое', 'молоко'],
  ['корень', 'имбиря'],
  ['ананас'],
  ['сок', 'лайма']],
 [['клубника'],
  ['манго'],
  ['сахар'],
  ['тертая', 'цедра', 'лайма'],
  ['текила'],
  ['сок', 'лайма'],
  ['апельсиновый', 'ликер']],
 [['манго'],
  ['темный', 'ром'],
  ['сок', 'лайма'],
  ['сахар'],
  ['тертая', 'цедра', 'лайма'],
  ['ананас']]]

### Стоп-слова и фильтрация

In [44]:
stopwords = nltk.corpus.stopwords.words('russian')

In [45]:
rx = re.compile(r'[a-zA-Zа-яА-Я][a-zA-Zа-яА-Я\d\-]*')

In [46]:
ingredients_filtered = (lens
    .Each().Each().modify(
        lambda tokenized: tuple(filter(
            lambda word: word not in stopwords and rx.match(word),
            tokenized))
    )(ingredients_tokenized)
)
ingredients_filtered[:4]

[[('бананы',),
  ('лимонный', 'сок'),
  ('молотая', 'корица'),
  ('ванильное', 'мороженое')],
 [('сахар',),
  ('кокосовое', 'молоко'),
  ('корень', 'имбиря'),
  ('ананас',),
  ('сок', 'лайма')],
 [('клубника',),
  ('манго',),
  ('сахар',),
  ('тертая', 'цедра', 'лайма'),
  ('текила',),
  ('сок', 'лайма'),
  ('апельсиновый', 'ликер')],
 [('манго',),
  ('темный', 'ром'),
  ('сок', 'лайма'),
  ('сахар',),
  ('тертая', 'цедра', 'лайма'),
  ('ананас',)]]

### Лемматизация

In [47]:
ingredients_lemmatized = lens.Each().Each().Each().modify(lemmatize)(ingredients_filtered)
ingredients_lemmatized[:4]

[[('банан',),
  ('лимонный', 'сок'),
  ('молотый', 'корица'),
  ('ванильный', 'мороженое')],
 [('сахар',),
  ('кокосовый', 'молоко'),
  ('корень', 'имбирь'),
  ('ананас',),
  ('сок', 'лайм')],
 [('клубника',),
  ('манго',),
  ('сахар',),
  ('тёртый', 'цедра', 'лайм'),
  ('текила',),
  ('сок', 'лайм'),
  ('апельсиновый', 'ликёр')],
 [('манго',),
  ('тёмный', 'ром'),
  ('сок', 'лайм'),
  ('сахар',),
  ('тёртый', 'цедра', 'лайм'),
  ('ананас',)]]

In [48]:
all_words_lemmatized = Counter(lens.Each().Each().Each().collect()(ingredients_lemmatized))

all_words_lemmatized_list = list(all_words_lemmatized)
all_words_lemmatized_list.sort()
all_words_lemmatized_list[:10]
# dict(all_words.items())

['абрикосовый',
 'ананас',
 'апельсиновый',
 'банан',
 'белок',
 'брынза',
 'ванилин',
 'ваниль',
 'ванильный',
 'вишня']

In [49]:
#ОЧЕНЬ ДОЛГО
fasttext.util.download_model('ru', if_exists='ignore')

'cc.ru.300.bin'

In [50]:
ft = fasttext.load_model('cc.ru.300.bin')
dim = ft.get_dimension()
dim

300

### Получаем векторы для слов 

In [51]:
word_embeddings_dict = {}
for word in all_words_lemmatized:
    word_embeddings_dict[word] = ft.get_word_vector(word)

In [52]:
first(word_embeddings_dict.items())

('банан',
 array([ 1.94887385e-01, -6.60526380e-02,  7.88492113e-02,  9.03176740e-02,
         8.94880146e-02, -1.48379147e-01,  2.24311762e-02, -4.75879619e-03,
         1.16267549e-02,  2.68354528e-02, -3.72944064e-02,  1.64822105e-03,
         5.10738790e-02,  1.01585977e-01,  6.48163483e-02, -1.15610115e-01,
        -3.05616856e-03,  3.19645479e-02, -6.46575615e-02, -6.13481402e-02,
        -3.28286402e-02,  7.68222064e-02,  2.55825445e-02,  7.40136206e-03,
         7.12529644e-02,  2.17049755e-02,  4.97756004e-02, -8.80596880e-03,
        -4.95500583e-03,  5.70715070e-02, -9.71776620e-02,  7.35110492e-02,
         2.72706505e-02,  3.32054943e-02, -4.71901185e-05, -1.14237860e-01,
        -2.79167667e-04, -9.27831605e-02, -6.35195374e-02, -5.84830642e-02,
         1.06558718e-01, -4.90855798e-03,  7.06979185e-02, -1.20925326e-02,
         6.83131441e-02,  5.63351624e-02, -1.07557401e-02,  7.19010383e-02,
         9.81174409e-02,  8.48399326e-02,  3.53266597e-02,  5.46764955e-02,
  

In [53]:
# word_embeddings_dict

In [54]:
# word_num = [(word, i) for i, word in enumerate(words)]
# word_num[:10]

### Векторы для ингредиентов

In [56]:
all_ingredients_list =  list(set(lens.Each().Each().collect()(ingredients_lemmatized)))
ingrediendts_numbering_dict = {ingredient: i for i, ingredient in enumerate(all_ingredients_list)}
ltake(6, ingrediendts_numbering_dict.items())

[(('сахар',), 0),
 (('круглый', 'рис'), 1),
 (('ванилин',), 2),
 (('яблоко',), 3),
 (('сок', 'лайм'), 4),
 (('пшеничный', 'мука'), 5)]

Для получения вектора для ингридиента просто суммируем векторы для каждого слова в этом ингридиенте:

In [57]:
ingrediendts_embeddings_dict = {}
for ingr in all_ingredients_list:
    ingrediendts_embeddings_dict[ingr] = np.zeros(dim)
    for word in ingr:
#         ingrediendts_embeddings_dict[ingr] += ft.get_word_vector(word) 
        ingrediendts_embeddings_dict[ingr] += word_embeddings_dict[word] # Должно заработать, когда 
                                                                         # ингредиенты будут лемматизированны
ltake(1, ingrediendts_embeddings_dict.items())[0]


(('сахар',),
 array([ 1.24340042e-01,  3.59124690e-02, -7.38940015e-02,  9.88699123e-03,
         1.72709465e-01, -4.79051322e-02,  1.03424087e-01,  4.76791933e-02,
        -4.22827750e-02, -1.54863410e-02, -5.53189702e-02,  1.72781460e-02,
        -6.17738925e-02,  1.01834489e-02,  9.77085438e-03, -3.08230519e-04,
        -2.22569741e-02, -6.93432316e-02, -1.42760873e-01, -9.81678367e-02,
        -8.66206735e-03,  8.64073932e-02, -4.00718451e-02, -1.65883396e-02,
         5.02421558e-02, -2.49747001e-03, -1.25526767e-02, -1.14722522e-02,
         5.36136068e-02,  4.65937369e-02,  9.07786191e-04, -6.85020760e-02,
         8.52591731e-03,  8.42294246e-02,  1.85608882e-02, -1.21564075e-01,
        -1.55991605e-02, -9.64115486e-02, -3.48125175e-02, -9.32337344e-02,
         1.77478164e-01,  2.10252292e-02,  5.76703846e-02, -4.81014047e-03,
         9.61419716e-02,  7.97744170e-02, -3.92269343e-02,  4.95163500e-02,
         1.44876957e-01,  1.19171273e-02,  6.95892274e-02,  4.64982819e-03,

Наконец, для каждого рецепта возьмем его векторизацию как сумму векторов его ингридиентов

In [112]:
ingredients_by_recipe_embedding = [
    sum(ingrediendts_embeddings_dict[ingredient]
        for ingredient in ingredient_list) 
    for ingredient_list in ingredients_lemmatized
]
len(ingredients_by_recipe_embedding)

[array([ 6.11611146e-01, -7.43011748e-02, -9.82487488e-02,  2.95895687e-01,
         6.62080204e-01, -3.13547595e-01,  1.76691884e-01,  2.96737695e-01,
        -3.21767492e-01, -9.82503085e-02, -6.69691958e-02,  1.11161237e-01,
         1.83635742e-01,  5.27145507e-01, -4.90509889e-02, -1.51326709e-01,
        -2.17026065e-01, -2.50674499e-01, -4.78544405e-01, -5.78503035e-01,
         2.25643182e-01,  1.19852779e-01, -1.30954622e-01, -1.71044090e-01,
         1.60740026e-01,  9.68547594e-02,  1.09688135e-01,  1.35400229e-01,
         1.56385407e-01,  1.27294251e-01, -4.84309057e-02, -1.17043031e-01,
         1.56720753e-01, -9.09083663e-03, -3.17208903e-02, -5.02322855e-01,
         6.18495652e-03, -7.03854598e-01, -4.14867880e-01, -4.76468764e-01,
         4.56838891e-01, -5.30690271e-02,  4.81506856e-01, -2.57825172e-02,
         1.66637813e-01,  5.37062930e-01,  1.70729541e-01,  1.22201854e-01,
         5.25353869e-01,  1.52826995e-01,  3.31789175e-01,  4.16002873e-01,
        -4.3

## Данные для нейросети

Всевозможные метки (course):

In [59]:
all_courses = {name: num for num, name in enumerate(df['course'][~df['course'].duplicated()])}
all_courses

{'Выпечка и десерты': 0,
 'Завтраки': 1,
 'Закуски': 2,
 'Напитки': 3,
 'Основные блюда': 4,
 'Паста и пицца': 5,
 'Салаты': 6,
 'Соусы и маринады': 7,
 'Супы': 8}

In [60]:
df_for_learning = pd.DataFrame(data={
    'id': df['id'][:20],
    'ingredient_embedding': ingredients_by_recipe_embedding,
    'course_mark': [all_courses[course] for course in df['course']][:20]
})
df_for_learning

,id,ingredient_embedding,course_mark
0,28195,"[0.611611146479845, -0.0743011748418212, -0.09...",0
1,28141,"[0.8909059994039126, -0.11706455610692501, -0....",0
2,27929,"[1.08976399153471, -0.4261779775843024, -0.389...",0
3,28192,"[1.1749160811305046, -0.5175759345293045, -0.3...",0
4,28463,"[0.6190221486613154, -0.14342674519866705, -0....",0
5,37902,"[0.5526944873854518, 0.37551341066136956, -0.2...",0
6,18536,"[0.554089829325676, 0.639475641772151, -0.5342...",0
7,35253,"[0.7957627102732658, 0.561975886579603, -0.673...",0
8,47082,"[0.4236516177188605, 0.25815030839294195, -0.1...",0
9,35173,"[0.5946541959419847, 0.9411030139308423, -0.54...",0


In [109]:
# tf.keras.preprocessing.text.one_hot(all_courses, len(all_courses))

(20, 9)

## Подготовка BoE

In [118]:
data = np.array(ingredients_by_recipe_embedding)
labels = to_categorical(np.array(df_for_learning['course_mark']), num_classes=len(all_courses))

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

VALIDATION_SPLIT = 0.2
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

## Обучение BoE

In [120]:
sequence_input = layers.Input(shape=(dim,), dtype='int32')
x = layers.Dense(128, activation='relu')(sequence_input)
x = layers.Dense(32, activation='relu')(x)
preds = layers.Dense(len(all_courses), activation='softmax')(x)

model = keras.Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

# happy learning!
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=2, batch_size=128)

Epoch 1/2
1/1 [==============================] - 1s 575ms/step - loss: 2.2628 - acc: 0.0000e+00 - val_loss: 1.8378 - val_acc: 0.5000
Epoch 2/2
1/1 [==============================] - 0s 44ms/step - loss: 1.9039 - acc: 0.3750 - val_loss: 1.4933 - val_acc: 0.7500


## Подготовка к CNN

In [124]:
for i in ingredients_lemmatized[:2]:
    print(list(map(lambda x: ingrediendts_numbering_dict[x], i)))

[8, 36, 11, 51]
[0, 61, 31, 70, 4]


In [ ]:
ingredients_lemmatized
data = np.array(ingredients_by_recipe_embedding)
labels = to_categorical(np.array(df_for_learning['course_mark']), num_classes=len(all_courses))


In [74]:
#Эта матрица будет использоваться для слоя Embedding в keras
ingrediendts_embeddings_matrix = np.zeros((len(all_ingredients_list), dim))
ingrediendts_embeddings_matrix.shape
for ingr in all_ingredients_list:
    ingrediendts_embeddings_matrix[ingrediendts_numbering_dict[ingr]] = ingrediendts_embeddings_dict[ingr]
# ingrediendts_embeddings_matrix[0]

## Прочее:

In [20]:
it = df.groupby('cuisine').count()
it.sort_values('title', ascending=False).head(30)

,id,title,course,ingredients
cuisine,,,,
Европейская кухня,11699,11699,11699,11699
Русская кухня,7111,7111,7111,7111
Итальянская кухня,4943,4943,4943,4943
Французская кухня,3055,3055,3055,3055
Американская кухня,2192,2192,2192,2192
Авторская кухня,1822,1822,1822,1822
Китайская кухня,840,840,840,840
Греческая кухня,640,640,640,640
Мексиканская кухня,583,583,583,583


In [44]:
a = ft.get_word_vector('кокосовый')-ft.get_word_vector('кокосовое')+ft.get_word_vector('кремовое')-ft.get_word_vector('кремовый')
a@a

0.5043752